In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# введите имя папки на гугл диске, в которую вы сохранили папку vision,
# содержащую .py файлы, а также папки datatasets и classifiers например,
# cv/assignments/assignment1/vision/
FOLDERNAME = None

assert FOLDERNAME is not None, "[!] Enter the foldername."

%cd drive/My Drive
%cp -r $FOLDERNAME ../../
%cd ../../
%cd vision/datasets/
!bash get_datasets.sh
%cd ../../

# Классификатор Softmax

*Выполните задания и сдайте их, сохранив полученные результаты, вместе со всем необходимым кодом, включая код за пределами этого ноутбука, например, код в папке `vision`*

Это задание аналогично заданию с SVM, и состоит из следующиъ частей:

- реализация векторной версии функции потерь для классификатора Softmax
- реализация векторной версии для аналитического градиента функции потерь
- проверка градиента с помощью градиента, полученного численным методом
- поиск значений для скорости обучения и коэффициента регуляризации с помощью кросс-валидации
- оптимизация функции потерь с помощью стохастического градиентного спуска
- визуализация параметров модели, полученных в результате обучения

In [ ]:
# Этот код нужно запустить для предватирельных настроек

import random
import numpy as np
from vision.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

# Чтобы графики выводились в том же окне, а не новом
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # размер изображений
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Загрузка изображений из выборки  CFAR-10
    cifar10_dir = 'vision/datasets/cifar-10-batches-py'
    
    # Очистка значений переменных, чтобы избежать дублирования данных при 
    # повторной загрузке
    try:
       del X_train, y_train
       del X_test, y_test
       print('Clear previously loaded data.')
    except:
       pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # Выбираем подмножество изображений
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Предобработка: трасформируем загруженные изображения в векторы
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Нормализация данных: вычтем среднее значнение пикселей в
    # выборке из каждого изображения
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # добавим измерения для коэффициента b и трасформируем векторы в колонки
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Выполним описанную выше функцию для того, чтобы получить данные
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

## Классификатор Softmax

Весь код для данного раздела должен быть написан в файле `vision/classifiers/softmax.py`.


In [ ]:
# Для начала напишите ту часть функции softmax_loss_naive function в файле 
# vision/classifiers/softmax.py, которая возвращает представляет собой наивную 
# реализацию функции потерь softmax с вложенными циклами. Градиент пока считать 
# не нужно

from vision.classifiers.softmax import softmax_loss_naive
import time

# Сгенерируем матрицу весов W, и посчитаем для нее значение функции потерь
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# Значение функции потерь должно быть равным примерно -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

**Вопрос**

Почему мы ожидаем, что значение функции потерь примерно равно -log(0.1)? Вспомните, что мы с вами обсуждали на лекции и дайте короткий отве в этой ячейке**

$\color{blue}{\textit Ответ:}$ *Напишите ваш ответ здесь* 



In [ ]:
# Домишите функцию softmax_loss_naive и реализуйте в ней вычисление градиента 
# наивным способом с использованием вложенных циклов
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# С помощью градиента, найденного численно, проверим нашу реализацию градиента
from vision.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# и проверим градиент с ненулевым коэффициентом регуляризации
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

In [ ]:
# Теперь реализуйте векторную версию функции потерь и градиента в функции 
# softmax_loss_vectorized. Результаты наивной и векторной реализации должны 
# совпадать, но векторная реализация должна работать быстрее
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from vision.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# С помощью нормы Фробениуса проверим, что обе версии градиента совпадают
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

In [ ]:
# Используте проверочную выборку, чтобы подобрать значения гиперпараметров - 
# коэффициент регуляризации и скорость обучения. Эксперементируйте с разными 
# диапазонами значений для сокрости обучения и регуляризации. При успешном 
# подборе гиперпараметров, точность распознавания должна равняться примерно 0,35

from vision.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None

################################################################################
# TODO:                                                                        #
# Подберите скорость обучения и коэффициент регуляризации с помощью            #
# проверочной выборки. Сохраните лушучю модель в переменной best_softmax.      #
################################################################################

# Эти значения - просто пример, вы можете их менять при необходимости
learning_rates = [1e-7, 5e-7]
regularization_strengths = [2.5e4, 5e4]

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

pass

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    
# Выведем результаты
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

In [ ]:
# Определим точность лучшего классификатора на тестовой выборке
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

In [ ]:
# Отобразим значения найденных параметров для каждого класса
w = best_softmax.W[:-1,:]
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Проведем масштабирование параметров так, чтобы значения были
    # в диапазоне от 0 до 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])

---
# Важно

Не забудьте сохранит результат